# Google Earth Engine Tutorial

This tutorial is borrowed from NCEAS: https://learning.nceas.ucsb.edu/2022-09-arctic/sections/15-google-earth-engine.html.

GEE Data catalogue is here: https://developers.google.com/earth-engine/datasets

More tutorial here: https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api

In [ ]:
# Dependencies
import ee
import geemap

In [ ]:
# triggers the authentication process
ee.Authenticate() 

In [ ]:
# Initialize
ee.Initialize() 

In [ ]:
# Create a leaflet map
myMap = geemap.Map(center = [10, 15], zoom = 2)
myMap

In [ ]:
# Gather ECMWF ERA5 Weather data collection
weatherData = ee.ImageCollection('ECMWF/ERA5/DAILY')

In [ ]:
# Check it out
print(type(weatherData))
weatherData

In [ ]:
# select desired bands (total_preciptation)
precip = weatherData.select("total_precipitation")
print(type(weatherData))

In [ ]:
# initial date of interest (inclusive)
i_date = '1995-01-01'

# final data of interest (exclusive)
f_date = '1996-01-01'

# select desired bands (total_preciptation), dates, and calculate mean precipitation across that date range
precip = weatherData.select("total_precipitation").filterDate(i_date, f_date).mean()

In [ ]:
# Check it out
print(type(precip))
precip

In [ ]:
# Make a color palet
precip_palette = {
    'min':0,
    'max':0.01,
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']
}

In [ ]:
# Plot it
myMap.addLayer(precip, precip_palette, 'total precipitation', opacity = 0.3)
myMap

In [ ]:
from IPython.display import Image

# Define the urban location of interest as a point near Lyon, France.
u_lon = 4.8148
u_lat = 45.7758
u_poi = ee.Geometry.Point(u_lon, u_lat)

# Define a region of interest with a buffer zone of 1000 km around Lyon.
roi = u_poi.buffer(1e6)

# Get a feature collection of administrative boundaries.
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')

# Filter the feature collection to subset France.
fr = countries.filter(ee.Filter.eq('ADM0_NAME', 'France'))

# Clip the image by France.
precip_fr = precip.clip(fr)

# Create the URL associated with the styled image data.
url = precip_fr.getThumbUrl({
    'min': 0, 'max': 0.01, 'region': roi, 'dimensions': 512,
    'palette': ['#FFFFFF', '#00FFFF', '#0080FF', '#DA00FF', '#FFA400', '#FF0000']})

# Display a thumbnail of elevation in France.
Image(url=url)

In [ ]:
# Get a link to download 
link = precip_fr.getDownloadURL({
    'scale': 30,
    'crs': 'EPSG:4326',
    'fileFormat': 'GeoTIFF',
    'region': u_poi})
print(link)

There are many datasets available in GEE. You can also leverage it to analyse e.g., Landsat

In [ ]:
# initial date of interest (inclusive)
i_date = '2022-01-01'

# final data of interest (exclusive)
f_date = '2022-02-01'

dataset = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(i_date, f_date)


# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )


dataset = dataset.map(apply_scale_factors)

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

m = geemap.Map()
m.set_center(-0.1870, 5.6037, 10)
m.add_layer(dataset, visualization, 'True Color (432)')
m